# Spotify recommendation system 

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

# Set your credentials
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="260e2fd648cf42dbab5157bb9857e07e",
    client_secret="aabb18f4c93440f7b115cfe963189e9f",
    redirect_uri="http://127.0.0.1:8888/callback",
    scope="user-library-read"
))
# Get saved tracks
results = sp.current_user_saved_tracks(limit=50)
tracks = []

In [2]:
while results:
    for item in results['items']:
        track = item['track']
        tracks.append({
            'track_name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date']
        })
    # Pagination
    if results['next']:
        results = sp.next(results)
    else:
        results = None

# Convert to DataFrame and save to CSV
df = pd.DataFrame(tracks)
df.to_csv('spotify_user_tracks.csv', index=False)

In [3]:
df.head()

,track_name,artist,album,release_date
0,Glue Song,beabadoobee,Glue Song,2023-02-14
1,At My Worst (feat. Kehlani),Pink Sweat$,PINK PLANET,2021-02-12
2,when the party's over,Billie Eilish,when the party's over,2018-10-17
3,Eyes Off You,PRETTYMUCH,Phases - EP,2019-05-24
4,blue,yung kai,blue,2024-08-02
